## Estudo do projeto ModeloPreditivoInadimplencia

Estudo baseado no projeto do canal https://www.youtube.com/@nerddosdados

# Preparação 

Preparando os pacotes que irão ser utilizados e carregando os dados do banco de dados.

In [ ]:
#Importação dos Pacotes
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import joblib # Utilizado para salvar o modelo preditivo
from sklearn.preprocessing import LabelEncoder #Utilizada para fazer o OneHotEncoding
from sklearn.metrics import mean_squared_error,precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix
from imblearn import under_sampling, over_sampling #Utilizada para fazer o balanceamento de dados
from imblearn.over_sampling import SMOTE #Utilizada para fazer o balanceamento de dados
from sklearn.preprocessing import MinMaxScaler #Utilizada para fazer a padronização dos dados
from sklearn.metrics import r2_score # Utilizado para medir a acuracia do modelo preditivo
import pymssql as sql #conexão SQL

warnings.filterwarnings("ignore") 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.options.display.float_format = '{:.2f}'.format

In [ ]:
#Importação dos arquivos direto do banco de dados MSSQL

# Cria a conexão com o SQL Server passando os parametros (Servidor, Usuário, Senha, Database)
conexao = sql.connect('localhost', 'usuario_python', '123456', 'MODELOS_PREDITIVOS')

# Chama a consulta ao banco de dados passando os parametros da conexão criada
df_original = pd.read_sql_query('select * from EXTRACAO_DADOS_SISTEMA', conexao)

# Fecha conexão com banco de dados
conexao.close()

## Análise

In [ ]:
#Tamanho do conjunto de dados. 
df_original.shape

In [ ]:
#Visão geral do conjunto de dados
df_original.head(5)

In [ ]:
# Analisando os dados missing podemos constatar o seguinte:
# Variavel DATA_ASSINATURA_CONTRATO possui 1 registro faltando
# Variavel VL_TOTAL_PC_PAGAS possui 4 registros faltando

# Como temos um total de 10.415 observações então excluir 5 observações NÃO afetará nosso trabalho

df_original.isnull().sum()

In [ ]:
#Informações básicas sobre tipos de variáveis
df_original.info(verbose=True)

In [ ]:
# Avaliar o período dos dados coletados
inicio = pd.to_datetime(df_original['DATA_ASSINATURA_CONTRATO']).dt.date.min()
fim = pd.to_datetime(df_original['DATA_ASSINATURA_CONTRATO']).dt.date.max()
print('Período dos dados - De:', inicio, 'Até:',fim)

In [ ]:
# Total de valores únicos de cada variável
# A variável TIPO_FINANCIAMENTO possui valor unico, então será retirada do nosso DataFrame

valores_unicos = []
for i in df_original.columns[0:20].tolist():
    print(i, ':', len(df_original[i].astype(str).value_counts()))
    valores_unicos.append(len(df_original[i].astype(str).value_counts()))

In [ ]:
# Visualizando algumas medidas estatisticas
df_original.describe()

In [ ]:
print('Maior Valor Financiado:', df_original['VALOR_FINANCIAMENTO'].max())
print('Menor Valor Financiado:', df_original['VALOR_FINANCIAMENTO'].min())

In [ ]:
# A variavel alvo precisará ser balanceada na etapa de pré-processamento
df_original.groupby(['INADIMPLENTE_COBRANCA']).size()

In [ ]:
# Aqui não precisaremos alterar nada
df_original.groupby(['PZ_FINANCIAMENTO']).size()

In [ ]:
# Aqui não precisaremos fazer nenhum tratamento
df_original.groupby(['RENDA_MENSAL_CLIENTE']).size()

In [ ]:
# Observe que temos uma variedade muito grande de valor financiado, neste caso devemos criar um range de valores
df_original.groupby(['VALOR_FINANCIAMENTO']).size()

# Tratamento

In [ ]:
# Excluindo os registros NA (5 registros)
df_original.dropna(inplace=True)

In [ ]:
# Criando faixa de prazos para utilizarmos no modelo preditivo
bins = [-100, 120, 180, 240]
labels = ['Até 120 Meses', '121 até 180 Meses', '181 até 240 Meses']
df_original['FAIXA_PRAZO_FINANCIAMENTO'] = pd.cut(df_original['PZ_FINANCIAMENTO'], bins=bins, labels=labels)
pd.value_counts(df_original.FAIXA_PRAZO_FINANCIAMENTO)

In [ ]:
# Criando faixa salarial para utilizarmos no modelo preditivo
bins = [-100, 100000, 200000, 300000, 400000, 500000, 750000, 1000000, 9000000000]
labels = ['Até 100 mil', '101 até 200 mil', '201 até 300 mil', '301 até 400 mil', '401 até 500 mil', 
          '501 até 750 mil', 'De 751 até 1.000.000','Mais de 1.000.000']
df_original['FAIXA_VALOR_FINANCIADO'] = pd.cut(df_original['VALOR_FINANCIAMENTO'], bins=bins, labels=labels)
pd.value_counts(df_original.FAIXA_VALOR_FINANCIADO)

In [ ]:
# Podemos retirar a variável TIPO FINANCIAMENTO como vimos anteriormente
# Devemos retirar a variável VALOR_FINANCIAMENTO pois criamos uma variavel de faixa de valores para ela.
# Devemos retirar a variável PRAZO_FINANCIAMENTO pois criamos uma variavel de faixa de meses para ela.
# Podemos retirar a variável DATA_ASSINATURA_CONTRATO
# Podemos retirar a variável NUMERO_CONTRATO

colunas = ['TAXA_AO_ANO', 'CIDADE_CLIENTE', 'ESTADO_CLIENTE','RENDA_MENSAL_CLIENTE', 
           'QT_PC_ATRASO', 'QT_DIAS_PRIM_PC_ATRASO','QT_TOTAL_PC_PAGAS',
           'VL_TOTAL_PC_PAGAS', 'QT_PC_PAGA_EM_DIA','QT_DIAS_MIN_ATRASO',
           'QT_DIAS_MAX_ATRASO', 'QT_DIAS_MEDIA_ATRASO','VALOR_PARCELA',
           'IDADE_DATA_ASSINATURA_CONTRATO', 'FAIXA_VALOR_FINANCIADO',
           'FAIXA_PRAZO_FINANCIAMENTO','INADIMPLENTE_COBRANCA']

df_dados = pd.DataFrame(df_original, columns=colunas)

In [ ]:
df_dados.head()

In [ ]:
df_dados.shape

In [ ]:
df_dados.info(verbose = True)

In [ ]:
df_dados.isnull().sum()

# Análise exploratória 

Nesta analise temos 2 objetivos: a variável alvo (INADIMPLENTE_COBRANCA), avaliar as variáveis categóricas para conhecimento dos dados e descartar variáveis que não fazem sentido.

In [ ]:
#Analisando como a variavel alvo está distribuida.
#Aqui podemos observar que há muito mais cotas como INADIMPLENTE
#dessa forma, precisaremos balancear o dataset mais adiante.
df_dados.INADIMPLENTE_COBRANCA.value_counts().plot(kind='bar', title='Inadimplentes',color = ['#1F77B4', '#FF7F0E']);


In [ ]:
#Podemos constatar na analise que não há discrepancias nestas variaveis

plt.rcParams["figure.figsize"] = [14.00, 3.50]
plt.rcParams["figure.autolayout"] = True
sns.countplot(data = df_dados, x = "FAIXA_VALOR_FINANCIADO", hue = "INADIMPLENTE_COBRANCA")
plt.show()

In [ ]:
#Podemos constatar na analise que não há discrepancias nestas variaveis

plt.rcParams["figure.figsize"] = [12.00, 3.50]
plt.rcParams["figure.autolayout"] = True
sns.countplot(data = df_dados, x = "FAIXA_PRAZO_FINANCIAMENTO", hue = "INADIMPLENTE_COBRANCA")
plt.show()

# Analise de Variaveis numericas

In [ ]:
df_dados.info()

In [ ]:
#carregar variaveis para plot
variaveis_numericas = []
for i in df_dados.columns[0:17].tolist():
        if df_dados.dtypes[i] == 'int64' or df_dados.dtypes[i] == 'float64':                        
            variaveis_numericas.append(i)    

In [ ]:
#Visualizando as variáveis numéricas
variaveis_numericas

In [ ]:
#Quantidade de variaveis
len(variaveis_numericas)

In [ ]:
#Podemos observar nos boxplots abaixo que as variáveis númericas apresentam uma grande quantidade de "possíveis" outliers
#Precisamos avaliar cada uma dessas variaveis dentro do contexto dos dados para saber se realmente iremos trata-las como outlier

plt.rcParams["figure.figsize"] = [14.00, 14.00]
plt.rcParams["figure.autolayout"] = True
f, axes = plt.subplots(4, 3) #4 linhas e 3 colunas

linha = 0
coluna = 0
for i in variaveis_numericas:
    sns.boxplot(data = df_dados, y=i, ax=axes[linha][coluna])
    coluna += 1
    if coluna == 3:
        linha += 1
        coluna = 0            

plt.show()

In [ ]:
# carregar variaveis categoricas para OneHotEncoding, ou seja transformar os campos categoricos em valores inteiros
# Colocamos o SLICE até 16 porque NÃO precisamos fazer OneHotEncoding para variavel TARGET
variaveis_categoricas = []
for i in df_dados.columns[0:16].tolist():
        if df_dados.dtypes[i] == 'object' or df_dados.dtypes[i] == 'category':                        
            variaveis_categoricas.append(i)    

In [ ]:
# Visualizando as variaveis categoricas
variaveis_categoricas

In [ ]:
# Cria o encoder e aplica OneHotEncoder
lb = LabelEncoder()

for var in variaveis_categoricas:
    df_dados[var] = lb.fit_transform(df_dados[var])

In [ ]:
df_dados.head()

In [ ]:
# Visualizando os tipos das variaveis
df_dados.info()

# Balanceamento

In [ ]:
# Visualiznado a quantidade da variavel target para balanceamento
variavel_target = df_dados.INADIMPLENTE_COBRANCA.value_counts()
variavel_target

In [ ]:
#Separar variaveis preditoras e target
PREDITORAS = df_dados.iloc[:, 0:15]  
TARGET = df_dados.iloc[:, 16] 

In [ ]:
# Visualizando as variaveis preditoras
PREDITORAS.head()

In [ ]:
# Visualizando a variavel target
TARGET.head()

In [ ]:
# Seed para reproduzir o mesmo resultado
seed = 100

# Cria o balanceador SMOTE
balanceador = SMOTE(random_state = seed)

# Aplica o balanceador
PREDITORAS_RES, TARGET_RES = balanceador.fit_resample(PREDITORAS, TARGET)

In [ ]:
# Visualizando o balanceamento da variável TARGET
plt.rcParams["figure.figsize"] = [12.00, 5.00]
plt.rcParams["figure.autolayout"] = True
TARGET_RES.value_counts().plot(kind='bar', title='Inadimplentes x Não Inadimplentes',color = ['#1F77B4', '#FF7F0E']);

In [ ]:
# Quantidade de registros antes do balanceamento
PREDITORAS.shape

In [ ]:
# Quantidade de registros antes do balanceamento
TARGET.shape

In [ ]:
# Quantidade de registros após do balanceamento
PREDITORAS_RES.shape

In [ ]:
# Quantidade de registros após do balanceamento
TARGET_RES.shape

# Separação e normalização dos dados / Criação e treino do modelo / Avaliação de métricas

In [ ]:
# Divisão em Dados de Treino e Teste.
X_treino, X_teste, Y_treino, Y_teste = train_test_split(PREDITORAS_RES, TARGET_RES, test_size = 0.3, random_state = 42)

In [ ]:
# Normalizando as Variáveis - Pré Processamento dos Dados
Normalizador = MinMaxScaler()
X_treino_normalizados = Normalizador.fit_transform(X_treino)
X_teste_normalizados = Normalizador.transform(X_teste)

In [ ]:
# visualizando a dimensão dos dados de treino
X_treino_normalizados.shape

In [ ]:
# visualizando os dados de treino normalizados
X_treino_normalizados

In [ ]:
# Criando o classificador com Random Forest
clf = RandomForestClassifier(n_estimators  = 300)

# Construção do modelo
clf = clf.fit(X_treino_normalizados, Y_treino)

In [ ]:
# Verificando a acuracia do modelo com dados de teste
scores = clf.score(X_teste_normalizados,Y_teste)
scores

In [ ]:
# Exibindo a importancia de cada variavel no modelo preditivo
plt.rcParams["figure.figsize"] = [10.00, 10.00]
plt.rcParams["figure.autolayout"] = True

importances = pd.Series(data=clf.feature_importances_, index=PREDITORAS.columns)
importances = importances.sort_values(ascending = False)
sns.barplot(x=importances, y=importances.index, orient='h').set_title('Importância de cada variável')
plt.show()

In [ ]:
importances.sort_values(ascending = False)

In [ ]:
# Salvando o modelo criado e treinado
joblib.dump(clf, 'modelo_treinado.pk')

# Preparação para produção

In [ ]:
# Carregando o modelo treinado
clf = joblib.load('modelo_treinado.pk')

In [ ]:
#Importação do arquivo de dados

# Cria a conexão com o SQL Server passando os parametros (Servidor, Usuário, Senha, Database)
conexao = sql.connect('localhost', 'usuario_python', '123456', 'MODELOS_PREDITIVOS')

# Chama a consulta ao banco de dados passando os parametros da conexão criada
df_original = pd.read_sql_query('select * from EXTRACAO_DADOS_SISTEMA', conexao)

# Fecha conexão com banco de dados
conexao.close()

In [ ]:
# Excluindo dados missing
df_original.dropna(inplace=True)


# Criando faixa de prazos para utilizarmos no modelo preditivo
bins = [-100, 120, 180, 240]
labels = ['Até 120 Meses', '121 até 180 Meses', '181 até 240 Meses']
df_original['FAIXA_PRAZO_FINANCIAMENTO'] = pd.cut(df_original['PZ_FINANCIAMENTO'], bins=bins, labels=labels)
pd.value_counts(df_original.FAIXA_PRAZO_FINANCIAMENTO)


# Criando faixa salarial para utilizarmos no modelo preditivo
bins = [-100, 100000, 200000, 300000, 400000, 500000, 750000, 1000000, 9000000000]
labels = ['Até 100 mil', '101 até 200 mil', '201 até 300 mil', '301 até 400 mil', '401 até 500 mil', 
          '501 até 750 mil', 'De 751 até 1.000.000','Mais de 1.000.000']
df_original['FAIXA_VALOR_FINANCIADO'] = pd.cut(df_original['VALOR_FINANCIAMENTO'], bins=bins, labels=labels)
pd.value_counts(df_original.FAIXA_VALOR_FINANCIADO)

columns = ['TAXA_AO_ANO', 'CIDADE_CLIENTE', 'ESTADO_CLIENTE','RENDA_MENSAL_CLIENTE', 
           'QT_PC_ATRASO', 'QT_DIAS_PRIM_PC_ATRASO','QT_TOTAL_PC_PAGAS',
           'VL_TOTAL_PC_PAGAS', 'QT_PC_PAGA_EM_DIA','QT_DIAS_MIN_ATRASO',
           'QT_DIAS_MAX_ATRASO', 'QT_DIAS_MEDIA_ATRASO','VALOR_PARCELA',
           'IDADE_DATA_ASSINATURA_CONTRATO', 'FAIXA_VALOR_FINANCIADO',
           'FAIXA_PRAZO_FINANCIAMENTO','INADIMPLENTE_COBRANCA',]

df_dados = pd.DataFrame(df_original, columns=columns)

# carregar variaveis categoricas para OneHotEncoding
variaveis_categoricas = []
for i in df_dados.columns[0:16].tolist():
        if df_dados.dtypes[i] == 'object' or df_dados.dtypes[i] == 'category':                        
            variaveis_categoricas.append(i) 

lb = LabelEncoder()

for var in variaveis_categoricas:
    df_dados[var] = lb.fit_transform(df_dados[var])



# Separar variaveis preditoras
PREDITORAS = df_dados.iloc[:, 0:15]          
    

# Fazendo a normalização dos dados    
Normalizador = MinMaxScaler()
dados_normalizados = Normalizador.fit_transform(PREDITORAS)

previsoes = clf.predict(dados_normalizados)
probabilidades = clf.predict_proba(dados_normalizados)
df_original['PREVISOES'] = previsoes
df_original['PROBABILIDADES'] = probabilidades[:, 1]
df_original.head()

In [ ]:
df_original.to_excel('df_original_com_probabilidades.xlsx', index = False)

# Criando nova tabela e inserindo os dados de predição no sql

In [ ]:
# Separando as colunas que serão utilizadas para inserção
columns = ['NUMERO_CONTRATO', 'PREVISOES', 'PROBABILIDADES']
df_conversao = pd.DataFrame(df_original, columns=columns)
df_conversao.head()

In [ ]:
import pymssql as sql
# Cria a conexão com o SQL Server passando os parametros (Servidor, Usuário, Senha, Database)
conexao = sql.connect('localhost', 'usuario_python', '123456', 'MODELOS_PREDITIVOS')

# Criando um cursor e executando um LOOP no DataFrame para fazer o INSERT no banco SQL Server

cursor = conexao.cursor()

for index,row in df_conversao.iterrows():
    sql = "INSERT INTO RESULTADOS_INTERMEDIARIO (NUMERO_CONTRATO, PREVISOES, PROBABILIDADES) VALUES (%s, %s, %s)"
    val = (row['NUMERO_CONTRATO'],row['PREVISOES'],row['PROBABILIDADES'])    
    cursor.execute(sql, val)
    conexao.commit()
    


print("Dados inseridos com sucesso no SQL")   

In [ ]:
import pymssql as sql
conexao = sql.connect('localhost', 'usuario_python', '123456', 'MODELOS_PREDITIVOS')
cursor = conexao.cursor()
cursor.execute('EXEC SP_INPUT_RESULTADOS_MODELO_PREDITIVO')    
conexao.commit()
conexao.close()

In [ ]:
import pymssql as sql
conexao = sql.connect('localhost', 'usuario_python', '123456', 'MODELOS_PREDITIVOS')
cursor = conexao.cursor()
cursor.execute('TRUNCATE TABLE RESULTADOS_INTERMEDIARIO')
conexao.commit()
conexao.close()